# Analyzing Performance at the 2025 SF Marathon: Terrain, Pace, and Heart Rate

This notebook explores my performance at the 2025 San Francisco Marathon. The data is pulled from Strava records of my 26.2-mile race. The focus of this exploratory analysis is to identify the relationship between course structure and terrain (elevation, distance segments, and aid stations) on performance metrics such as pace and heartrate. The goal is to draw insights to guide future training and serve as a reference if I run this marathon agian. 

The analysis is done using SQL and Python (pandas, matplotlib) in a Jupyter notebook. Raw data (TCX and GPX) was cleaned in separate Python scripts. 

## 1. Overview of the Data 

In [ ]:
# Data, SQL, visualization imports
import sqlite3
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
import statsmodels.api as sm
import matplotlib.pyplot as plt
from geopy.distance import geodesic
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
# Connect to SQLite database, display tables and columns
con = sqlite3.connect("../data/marathon_data.db") 
cur = con.cursor()

# Table overview
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", con)
print("Tables:", tables['name'].tolist())

# Show structure for each table
for table in tables['name']:
    print(f"\n=== {table.upper()} ===")
    cols = pd.read_sql(f"PRAGMA table_info({table});", con)
    for _, row in cols.iterrows():
        print(f"  {row['name']}: {row['type']}")

The database contains three tables: `tcx_data`, `official_route_data`, and `aid_stations_data`. The names and data types of the columns in each are as follows:

- `tcx_data`: data from my race recorded on my Apple Watch
    - __time__: timestamp in UTC for each row entry; TEXT (convertible to date-time)
    - __latitude__: GPS latitude in decimal degrees (WGS84 coordinate system); REAL
    - __longitude__: GPS longitude in decimal degrees (WGS84 coordinate system); REAL
    - __elevation__: altitude above sea level, in meters; REAL
    - __distance__: cumulative distance covered since the start, in meters; REAL
    - __heart_rate__: runner’s heart rate at that moment, in beats per minute (BPM); INTEGER

- `official_route_data`: official route data provided by SFM organizers
    - __latitude__: GPS latitude in decimal degrees (WGS84 coordinate system); REAL
    - __longitude__: GPS longitude in decimal degrees (WGS84 coordinate system); REAL
    - __elevation__: altitude above sea level, in meters; REAL

- `aid_stations_data`: official route data provided by SFM organizers
    - __name__: name of the waypoint; TEXT
    - __latitude__: GPS latitude in decimal degrees (WGS84 coordinate system); REAL
    - __longitude__: GPS longitude in decimal degrees (WGS84 coordinate system); REAL
    - __type__: type of the waypoint; TEXT


## 2. Data cleaning and processing

Before analysis, the raw data was inspected and adjusted to improve consistency and interpretability:

- **Distance units:** Converted from meters to kilometers for readability.  
- **Route validation:** Compared the recorded elevation profile against the official Strava course to confirm alignment.  
- **Official course distance:** Since the official data only contained coordinates and elevation, distance in kilometers was calculated in Python for direct comparison.  
- **Data trimming:** Rows prior to the official start line were removed to ensure analyses begin from the race’s official start time.  
- **Elevation cleaning:** Short-term GPS anomalies and outliers (including a ~250 m spike) were identified using a rolling-baseline method. Points that deviated substantially from the local trend were marked as missing and then filled via linear interpolation, preserving the overall hill profile while removing unrealistic spikes.  
- **Split calculation:** Per-kilometer average pace and heart rate were computed by aggregating data into kilometer splits, providing a clearer view of discrete segment performance patterns.
- **Rolling averages:** A **rolling 1km pace** was derived by calculating elapsed time over the preceding kilometer of distance, smoothing out short-term fluctuations and approximating how fitness apps typically present "live pace." Similarly, a **rolling 30-second heart rate** (smoothed HR over a 30-second time window) was calculated to analyze cardiovascular response patterns throughout the race.
- **Course “valves,” aid stations, and truncated recording:** Certain sections of the route were temporarily rerouted for traffic or safety, moving runners slightly off the main path while keeping the total distance essentially unchanged. Aid stations were not always precisely at their official locations. The watch stopped recording at the exact marathon distance before the finish line, which slightly truncates the final segment. These factors can cause minor GPS offsets or pace fluctuations and explain why the recorded course map may differ slightly from the official route.  


In [ ]:
# Query tcx data with pre-processing

# Query tcx race data with distance in km
tcx_query = """
SELECT
    time,
    latitude,
    longitude,
    elevation,
    distance,
    distance / 1000.0 AS distance_km,
    heart_rate
FROM tcx_data 
WHERE (
    time >= '2025-07-27 12:35:29'
    AND elevation <= 100
)
ORDER BY time
"""

tcx_df = pd.read_sql(tcx_query, con)

# Create 1km splits with pace calculation in SQL
km_split_query = """
WITH km_split_table AS (
    SELECT
        CAST(FLOOR(distance / 1000.0) AS INT) AS km_split,
        MIN(time) AS start_time,
        MAX(time) AS end_time,
        MIN(distance) AS start_distance,
        MAX(distance) AS end_distance,
        AVG(elevation) AS avg_elevation,
        AVG(heart_rate) AS avg_heart_rate
    FROM tcx_data
    WHERE time >= '2025-07-27 12:35:29'
    GROUP BY km_split
)
SELECT
    km_split,
    start_time,
    end_time,
    start_distance,
    end_distance,
    (end_distance - start_distance) / 1000.0 AS actual_distance_km,
    avg_elevation,
    avg_heart_rate,
    (strftime('%s', end_time) - strftime('%s', start_time)) AS time_diff_seconds,
    CASE 
        WHEN (end_distance - start_distance) > 0 
        THEN ((strftime('%s', end_time) - strftime('%s', start_time)) / 60.0) / ((end_distance - start_distance) / 1000.0)
        ELSE NULL 
    END AS pace
FROM km_split_table
ORDER BY km_split
"""

km_split_df = pd.read_sql(km_split_query, con)

# Format pace in Python 
def format_pace_value(pace):
    if pace is None or np.isnan(pace):
        return "N/A"
    minutes = int(pace)
    seconds = int(round((pace - minutes) * 60))
    return f"{minutes}:{seconds:02d}"

km_split_df['pace_formatted'] = km_split_df['pace'].apply(format_pace_value)

km_split_df.tail()

In [ ]:
# Load and process official route data
def calculate_route_distances(route_df):
    """Calculate cumulative distances for route coordinates."""
    distances = [0.0]
    for i in range(1, len(route_df)):
        start_coords = (route_df.iloc[i-1]['latitude'], route_df.iloc[i-1]['longitude'])
        end_coords = (route_df.iloc[i]['latitude'], route_df.iloc[i]['longitude'])
        distances.append(geodesic(start_coords, end_coords).km)
    
    route_df = route_df.copy()
    route_df['segment_distance'] = distances
    route_df['cumulative_distance_km'] = route_df['segment_distance'].cumsum()
    return route_df

def map_stations_to_route(stations_df, route_df):
    """Map aid stations to their closest points on the official route."""
    def find_closest_route_distance(station_lat, station_lon):
        station_coords = (station_lat, station_lon)
        distances_to_route = [
            geodesic(station_coords, (row['latitude'], row['longitude'])).meters 
            for _, row in route_df.iterrows()
        ]
        closest_idx = np.argmin(distances_to_route)
        return route_df.iloc[closest_idx]['cumulative_distance_km']
    
    stations_df = stations_df.copy()
    stations_df['distance_from_start_km'] = stations_df.apply(
        lambda row: find_closest_route_distance(row['latitude'], row['longitude']), 
        axis=1
    )
    return stations_df.sort_values('distance_from_start_km').reset_index(drop=True)

# Load and process data
official_route_df = pd.read_sql("SELECT * FROM official_route_data;", con)
official_route_df = calculate_route_distances(official_route_df)

aid_stations_df = pd.read_sql("SELECT * FROM aid_stations_data;", con)
aid_stations_df = map_stations_to_route(aid_stations_df, official_route_df)

In [ ]:
# Create plots to compare my race to official route
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Elevation Comparison', 'Route Map Comparison'),
    horizontal_spacing=0.15,
    specs=[[{"secondary_y": False}, {"secondary_y": False}]]
)

# First subplot: Elevation comparison (left)
fig.add_trace(
    go.Scatter(
        x=tcx_df['distance_km'],
        y=tcx_df['elevation'],
        mode='lines',
        name='My Race',
        line=dict(color='blue', width=2),
        opacity=0.7,
        hovertemplate='Distance: %{x:.2f} km<br>My Elevation: %{y:.1f} m<extra></extra>'
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=official_route_df['cumulative_distance_km'],
        y=official_route_df['elevation'],
        mode='lines',
        name='Official Route',
        line=dict(color='red', width=2, dash='dash'),
        opacity=0.7,
        hovertemplate='Distance: %{x:.2f} km<br>Official Elevation: %{y:.1f} m<extra></extra>'
    ),
    row=1, col=1
)

# Second subplot: Route map (right)
fig.add_trace(
    go.Scatter(
        x=tcx_df['longitude'],
        y=tcx_df['latitude'],
        mode='lines',
        name='My Race Route',
        line=dict(color='blue', width=3),
        opacity=0.7,
        hovertemplate='Lat: %{y:.5f}<br>Lon: %{x:.5f}<br>My Route<extra></extra>',
        showlegend=False
    ),
    row=1, col=2
)

fig.add_trace(
    go.Scatter(
        x=official_route_df['longitude'],
        y=official_route_df['latitude'],
        mode='lines',
        name='Official Route',
        line=dict(color='red', width=2, dash='dash'),
        opacity=0.7,
        hovertemplate='Lat: %{y:.5f}<br>Lon: %{x:.5f}<br>Official Route<extra></extra>',
        showlegend=False
    ),
    row=1, col=2
)

# Add aid stations to map (right)
fig.add_trace(
    go.Scatter(
        x=aid_stations_df['longitude'],
        y=aid_stations_df['latitude'],
        mode='markers',
        name='Aid Stations',
        marker=dict(color='green', size=8, symbol='circle'),
        hovertemplate='%{text}<br>Lat: %{y:.5f}<br>Lon: %{x:.5f}<extra></extra>',
        text=aid_stations_df['name'] if 'name' in aid_stations_df.columns else 'Aid Station',
        showlegend=False
    ),
    row=1, col=2
)

# Update layout and axis labels
fig.update_layout(
    title_text='Marathon Analysis: Elevation and Route Comparison',
    height=500,
    width=1200,
    showlegend=True
)
fig.update_xaxes(title_text="Distance (km)", row=1, col=1)
fig.update_xaxes(title_text="Longitude", row=1, col=2)
fig.update_yaxes(title_text="Elevation (m)", row=1, col=1)
fig.update_yaxes(title_text="Latitude", row=1, col=2, scaleanchor="x2", scaleratio=1)

fig.show()

In [ ]:
# Remove elevation outliers using rolling baseline and interpolation
baseline_window = 50
tcx_df['elev_baseline'] = tcx_df['elevation'].rolling(window=baseline_window, center=True, min_periods=1).mean()

deviation_threshold = 5  # meters
tcx_df.loc[(tcx_df['elevation'] - tcx_df['elev_baseline']).abs() > deviation_threshold, 'elevation'] = np.nan
tcx_df['elevation'] = tcx_df['elevation'].interpolate()
tcx_df.drop(columns=['elev_baseline'], inplace=True)

# Create elevation comparison plot
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=tcx_df['distance_km'],
    y=tcx_df['elevation'],
    mode='lines',
    name='My Race Elevation',
    line=dict(color='blue', width=2),
    hovertemplate='Distance: %{x:.2f} km<br>Elevation: %{y:.1f} m<extra></extra>'
))

fig.add_trace(go.Scatter(
    x=official_route_df['cumulative_distance_km'],
    y=official_route_df['elevation'],
    mode='lines',
    name='Official Route Elevation',
    line=dict(color='red', width=2, dash='dash'),
    opacity=0.7,
    hovertemplate='Distance: %{x:.2f} km<br>Official Elevation: %{y:.1f} m<extra></extra>'
))

fig.update_layout(
    title='Elevation Profile: Recorded vs Official Route (After Outlier Removal)',
    xaxis_title='Distance (km)',
    yaxis_title='Elevation (m)',
    width=900,
    height=500,
    hovermode='x'
)

fig.show()


In [ ]:
# Find rolling pace and heart rate for cleaner visualization

# Find rolling average pace over 1 km
dist_m = tcx_df['distance'].to_numpy()
tcx_df['time'] = pd.to_datetime(tcx_df['time'])

time_s = tcx_df['time'].astype('int64') // 1_000_000_000  # seconds since epoch

pace_1km = np.full(len(dist_m), np.nan)

window_m = 1000  # 1 km

for i in range(len(dist_m)):
    target_dist = dist_m[i] - window_m
    start_idx = np.searchsorted(dist_m, target_dist, side='right') - 1
    start_idx = max(0, start_idx)
    
    dt = time_s[i] - time_s[start_idx]
    dx_km = (dist_m[i] - dist_m[start_idx]) / 1000
    
    if dx_km > 0:
        pace_1km[i] = (dt / 60) / dx_km  # min/km

tcx_df['pace_1km'] = pace_1km
tcx_df['pace_1km_str'] = tcx_df['pace_1km'].apply(format_pace_value)


# Find rolling heart rate over 30 seconds
# Remove duplicate timestamps, keeping the first occurrence
tcx_df_clean = tcx_df.drop_duplicates(subset='time', keep='first')
tcx_df_temp = tcx_df_clean.set_index('time')
hr_30s_clean = (
    tcx_df_temp['heart_rate']
    .rolling('30s', min_periods=5, center=True)
    .mean()
)

# Map back to original DataFrame
tcx_df['hr_30s'] = tcx_df['time'].map(hr_30s_clean.to_dict())

tcx_df.head()


## 3. Overall Trends

The following graph illustrates my rolling pace (1km window) and elevation across the 26.2-mile race. 

In [ ]:
# Create dual-axis plot: pace and elevation vs distance
fig = go.Figure()

# Add pace trace (1km rolling average)
fig.add_trace(go.Scatter(
    x=tcx_df['distance_km'],
    y=tcx_df['pace_1km'],
    mode='lines',
    name='Pace (1km rolling avg)',
    line=dict(color='red', width=2),
    hovertemplate='Distance: %{x:.2f}km<br>Pace: %{customdata}<extra></extra>',
    customdata=tcx_df['pace_1km_str'],
    yaxis='y'
))

# Add elevation trace
fig.add_trace(go.Scatter(
    x=tcx_df['distance_km'],
    y=tcx_df['elevation'],
    mode='lines',
    name='Elevation (m)',
    line=dict(color='blue', width=1),
    opacity=0.7,
    hovertemplate='Distance: %{x:.2f}km<br>Elevation: %{y:.0f}m<extra></extra>',
    yaxis='y2'
))

# Add aid stations with labels
for i, (_, station) in enumerate(aid_stations_df.iterrows()):
    # Vertical line
    fig.add_vline(
        x=station['distance_from_start_km'],
        line=dict(color='green', width=2, dash='dot'),
        opacity=0.7
    )
    
    # Distance label
    fig.add_annotation(
        x=station['distance_from_start_km'],
        y=1, yref="paper",
        text=f"{station['distance_from_start_km']:.1f}",
        showarrow=False,
        font=dict(size=10, color='green'),
        yanchor="bottom"
    )

# Add legend entry for aid stations
fig.add_trace(go.Scatter(
    x=[None], y=[None],
    mode='lines',
    name='Aid Stations',
    line=dict(color='green', width=2, dash='dot'),
    showlegend=True
))

# Update layout
fig.update_layout(
    title='Marathon Performance: Pace vs Elevation Profile with Aid Stations',
    xaxis_title='Distance (km)',
    yaxis=dict(title='Pace (min/km)', side='left'),
    yaxis2=dict(title='Elevation (m)', side='right', overlaying='y'),
    width=1000,
    height=500,
    hovermode='x unified',
    legend=dict(orientation="h", yanchor="top", y=-0.15, xanchor="center", x=0.5)
)

fig.show()

The visualization above highlights several key performance patterns:

- **Start-line anomaly:** An initial pace spike appears within the first 0.2km, reflecting bottleneck congestation near the starting line. 
- **Impact of terrain:** Pace strongly correlates with elevation gain/loss. Uphill segments produce sustained slowdowns, while downhill segments allow partial recovery. 
- **Effects of fatigue:** Although earlier downhill sections produce clear accelerations, the effect diminishes in the latter half of the race. This suggests that fatigue not only leads to slower pace but also reduces responsiveness to favorable terrain (e.g., knee stress reducing ability to capitalize on downhills). 
- **End-of-race acceleration:** Despite late-race fatigue, the last 5km show a distinct negative split. This shows a clear end-of-race push as the finish line draws closer. 




## . Heart Rate vs Pace Analyis

In [ ]:
# Create dual-axis plot: pace and heart rate vs distance
fig = go.Figure()

# Add pace trace (1km rolling average)
fig.add_trace(go.Scatter(
    x=tcx_df['distance_km'],
    y=tcx_df['pace_1km'],
    mode='lines',
    name='Pace (min/km; 1km rolling avg)',
    line=dict(color='red', width=2),
    hovertemplate='Distance: %{x:.2f}km<br>Pace: %{customdata}<extra></extra>',
    customdata=tcx_df['pace_1km_str'],
    yaxis='y'
))

# Add elevation trace
fig.add_trace(go.Scatter(
    x=tcx_df['distance_km'],
    y=tcx_df['hr_30s'],
    mode='lines',
    name='Heart Rate (bpm; 30s rolling avg)',
    line=dict(color='blue', width=1),
    opacity=0.7,
    hovertemplate='Distance: %{x:.2f}km<br>Heart Rate: %{y:.0f}bpm<extra></extra>',
    yaxis='y2'
))


# Update layout
fig.update_layout(
    title='Marathon Performance: Pace vs Heart Rate Profile',
    xaxis_title='Distance (km)',
    yaxis=dict(title='Pace (min/km)', side='left'),
    yaxis2=dict(title='Heart Rate (bpm)', side='right', overlaying='y'),
    width=1000,
    height=500,
    hovermode='x unified',
    legend=dict(orientation="h", yanchor="top", y=-0.15, xanchor="center", x=0.5)
)

fig.show()

The line graph above shows some mey insights:

- **HR vs. Pace trends**: Heart rate remains centered around ~150 bpm, while pace shows a gradual slowdown.  
- **Drift over time**: Early race = high HR + fast pace. Late race = high HR + slower pace, suggesting fatigue.  
- **End-phase variability**: More jitter near the end due to alternating walk/run efforts.  
- **Finish surge**: HR spikes again with the final acceleration at the race finish.  


In [ ]:
# Calculate Pearson correlation between pace and heart rate
valid_data = km_split_df.dropna(subset=['pace', 'avg_heart_rate'])
correlation_coef, p_value = pearsonr(valid_data['pace'], valid_data['avg_heart_rate'])
r_squared = correlation_coef ** 2

print(f"Pearson Correlation Analysis:")
print(f"Correlation coefficient (r): {correlation_coef:.3f}")
print(f"P-value: {p_value:.6f}")
print(f"R-squared: {r_squared:.3f}")

fig = go.Figure(go.Scatter(
    x=km_split_df['pace'],
    y=km_split_df['avg_heart_rate'],
    mode='markers', 
    marker=dict(
        size=10,
        color=km_split_df['km_split'],
        colorscale='Viridis',
        showscale=True,
        colorbar=dict(title="Distance (km)")
    ),
    hovertemplate=(
        "KM: %{marker.color}<br>"
        "Pace: %{customdata}<br>"
        "HR: %{y:.0f} bpm<extra></extra>"
    ),
    customdata=km_split_df['pace_formatted'] 
))

fig.update_layout(
    title=f"HR vs Pace (1 km splits)<br><sub>Pearson r = {correlation_coef:.3f}, R² = {r_squared:.3f}, p = {p_value:.4f}</sub>",
    xaxis_title="Pace (min/km)",
    yaxis_title="Heart Rate (bpm)",
    width=900,
    height=600
)

fig.show()

## . Heatmap

In [ ]:
# Enhanced cleanup: Close connections and clear variables
if 'con' in globals():
    con.close()
    print("Database connection closed successfully.")
else:
    print("No active database connection found.")

# Clear all user variables (keeping built-ins and IPython variables)
for var in list(globals().keys()):
    if not var.startswith('_') and var not in ['In', 'Out', 'exit', 'quit', 'get_ipython']:
        del globals()[var]

# Clear large variables from memory
import gc
gc.collect()
print("All variables cleared from memory.")
print("Kernel is still active and ready for new analysis.")